# Unsupervised Anomaly Detection on fastMRI

### *Run these cells only when in Google Colab*

In [1]:
!wget https://syncandshare.lrz.de/dl/fiNRBX6FN1Vh67NqugzqKo/data.zip
!unzip -q ./data.zip

--2023-07-19 00:30:07--  https://syncandshare.lrz.de/dl/fiNRBX6FN1Vh67NqugzqKo/data.zip
Resolving syncandshare.lrz.de (syncandshare.lrz.de)... 129.187.255.213
Connecting to syncandshare.lrz.de (syncandshare.lrz.de)|129.187.255.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40922332 (39M) [application/x-zip-compressed]
Saving to: ‘data.zip’

data.zip            100%[===================>]  39.03M   310KB/s    in 2m 16s  

2023-07-19 00:32:25 (294 KB/s) - ‘data.zip’ saved [40922332/40922332]



### *Install requirements*

In [2]:
!pip install -r requirements_gpu.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.4/722.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.6 MB/s eta 0:00:00


## Imports

In [3]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")
import pytorch_lightning as pl
import torch
import yaml
import plotly.graph_objects as go
import numpy as np

from model import PatchCoreModel

# autoreload imported modules
%load_ext autoreload
%autoreload 2

## Load the config

In [4]:
with open('./configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Reproducibility
pl.seed_everything(config['seed'])

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

## Check device

In [5]:
on_gpu = torch.cuda.is_available()
print(on_gpu)

True


## Prepare the model

In [6]:
model = PatchCoreModel(
    seed=config['seed'],
    split_dir=config['split_dir'],
    target_size=config['target_size'],
    batch_size=config['batch_size'],
    sampling_percentage=config['sampling_percentage'],
    backbone_names=config['backbone_ensemble'],
    layers_to_extract_from=config['layers_ensemble_bb'],
    neighbour_num=config['neighbour_num'],
    pathologies=config['pathologies'],
    plot_pathology=config['plot_pathology'],
    test_img_path=config['resection_test_image_paths'],
    test_mask_path=config['resection_test_mask_paths']
)

## Train the model

In [7]:
model.train()

Using 711 IXI images and 130 fastMRI images for training. Using 15 images for validation.


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Wide_ResNet101_2_Weights.IMAGENET1K_V1`. You can also use `weights=Wide_ResNet101_2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth" to /root/.cache/torch/hub/checkpoints/wide_resnet101_2-32ee1156.pth
100%|██████████| 243M/243M [00:15<00:00, 16.8MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated 

Training model wideresnet101 (1/3)



Subsampling...: 100%|██████████| 21529/21529 [00:35<00:00, 608.89it/s]


Training model resnext101 (2/3)



Subsampling...: 100%|██████████| 21529/21529 [00:37<00:00, 580.96it/s]


Training model densenet201 (3/3)



Subsampling...: 100%|██████████| 21529/21529 [00:38<00:00, 564.37it/s]


training complete


## Evaluate the model

In [8]:
results_collect = model.test()

Testing on disease:  absent_septum


Testing on disease:  artefacts


Testing on disease:  craniatomy


Testing on disease:  dural


Testing on disease:  ea_mass


...

Testing on disease:  posttreatment


Testing on disease:  resection


Testing on disease:  sinus


Testing on disease:  wml


Testing on disease:  other


## Print chosen scores

## *F1 Scores*

In [14]:
for i,p in enumerate(config['pathologies']):
  print(p, np.mean(results_collect[i]["pixel_f1"]))

absent_septum 0.6975717434283683
artefacts 0.13187198445535298
craniatomy 0.1191458065061523
dural 0.3662225538710157
ea_mass 0.22071035139958303
edema 0.32257058579843007
encephalomalacia 0.6110183634416794
enlarged_ventricles 0.4436646250175046
intraventricular 0.7613469980362237
lesions 0.10802226296873671
mass 0.2015865126119261
posttreatment 0.09307250953752105
resection 0.2894551498994228
sinus 0.022374934557482892
wml 0.15774931018541555
other 0.26952617624695385


## *Full Pixel AUROC*

In [12]:
for i,p in enumerate(config['pathologies']):
  print(p, np.mean(results_collect[i]["full_pixel_auroc"]))

absent_septum 0.9938138710274934
artefacts 0.8141021915099436
craniatomy 0.7953374447510114
dural 0.816499154035703
ea_mass 0.9482810679596494
edema 0.8763472141766787
encephalomalacia 0.9499094352743671
enlarged_ventricles 0.9402624352855339
intraventricular 0.9871902216120263
lesions 0.9177658304167925
mass 0.9667672831829378
posttreatment 0.7765881297298923
resection 0.9286457257395365
sinus 0.6228365281973547
wml 0.8862966503287985
other 0.9012789413105207


## Visualize the scores

In [10]:
fig_bp = go.Figure()
x = []
y = []
for i in range(len(config["pathologies"])):
  dataset_name = config["pathologies"][i]
  for dataset_val in results_collect[i]["pixel_f1"]:
      y.append(dataset_val)
      x.append(dataset_name)

fig_bp.add_trace(go.Box(
    y=y,
    x=x,
    name="F1",
    boxmean='sd'
))
title = 'score'
fig_bp.update_layout(
    yaxis_title=title,
    boxmode='group',  # group together boxes of the different traces for each value of x
    yaxis=dict(range=[0, 1]),
)
fig_bp.update_yaxes(range=[0, 1], title_text='F1 Score', tick0=0, dtick=0.1, showgrid=False)